In [ ]:
from comet_ml import Experiment
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from utils import *

EXP_NAME = "Logistic Regression"
EPOCHS = 1000

## Read Dataset from CSV file

CSV file is modified to have columns as follows:

- sentence: original sentence
- label : class (0 to 4)
- tokens : tokenized sentence
- score : sentiment score (0 to 1)


In [ ]:
train_df = pd.read_csv("dataset/train.csv")
print("Train size: ", len(train_df))

val_df = pd.read_csv("dataset/val.csv")
print("Val size  : ", len(val_df))

test_df = pd.read_csv("dataset/test.csv")
print("Test size : ", len(test_df))

## Classes Distribution


In [ ]:
print("Train class distribution:")
print(train_df.label.value_counts().sort_index())

print("Val class distribution:")
print(val_df.label.value_counts().sort_index())

print("Test class distribution:")
print(test_df.label.value_counts().sort_index())

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
train_df["label"].value_counts().sort_index().plot(kind="bar", color="skyblue")
plt.title("Train Dataset Class Distribution")
plt.xlabel("Class")
plt.ylabel("Frequency")

plt.subplot(1, 3, 2)
val_df["label"].value_counts().sort_index().plot(kind="bar", color="lightgreen")
plt.title("Val Dataset Class Distribution")
plt.xlabel("Class")
plt.ylabel("Frequency")

plt.subplot(1, 3, 3)
test_df["label"].value_counts().sort_index().plot(kind="bar", color="salmon")
plt.title("Test Dataset Class Distribution")
plt.xlabel("Class")
plt.ylabel("Frequency")

plt.tight_layout()
plt.show()

## Tokenization


In [ ]:
def tokenize(tokens):
    return tokens.lower().split()

In [ ]:
print("Sentence:")
print(train_df.iloc[3]["sentence"])

print("\nSentence Tokenized:")
tokens = tokenize(train_df.iloc[3]["sentence"])
print(tokens)

## Bigrams


In [ ]:
def extract_bigrams(sentence):
    words = tokenize(sentence)
    return [(words[i], words[i + 1]) for i in range(len(words) - 1)]

## Feature Matrix for Train dataset

using int8 instead of float64 to save memory


In [ ]:
def generate_train_features(dataframe):
    unique_bigrams = set()
    for _, sample in dataframe.iterrows():
        sample_bigrams = extract_bigrams(sample["sentence"])
        unique_bigrams.update(sample_bigrams)

    bigrams_indices = {bi_gram: index for index, bi_gram in enumerate(unique_bigrams)}

    feature_matrix = np.zeros((len(dataframe), len(unique_bigrams))).astype(np.int8)

    for i, (_, sample) in enumerate(dataframe.iterrows()):
        sample_bigrams = extract_bigrams(sample["sentence"])
        for b in sample_bigrams:
            index = bigrams_indices.get(b)
            if index is not None:
                feature_matrix[i, index] = 1

    return feature_matrix, bigrams_indices

In [ ]:
train_feature_matrix, train_bigrams_indices = generate_train_features(train_df)
print(
    "Train Feature matrix shape (samples x unique bigrams):", train_feature_matrix.shape
)
print("Train Feature matrix datatype:", train_feature_matrix.dtype)

## Unify features for val & test set

Remove all Features (bigrams) from val & test set that doesn't appear in training features


In [ ]:
def filter_features(dataframe):
    unique_dataframe_bigrams = set()
    for _, sample in dataframe.iterrows():
        sample_bigrams = extract_bigrams(sample["sentence"])
        unique_dataframe_bigrams.update(sample_bigrams)

    unique_train_bigrams = set(train_bigrams_indices.keys())
    common_bigrams = unique_dataframe_bigrams.intersection(unique_train_bigrams)
    feature_matrix = np.zeros((len(dataframe), len(train_bigrams_indices))).astype(
        np.int8
    )

    for i, (_, sample) in enumerate(dataframe.iterrows()):
        sample_bigrams = extract_bigrams(sample["sentence"])
        for b in sample_bigrams:
            if b in common_bigrams:
                index = train_bigrams_indices.get(b)
                feature_matrix[i, index] = 1

    return feature_matrix

In [ ]:
val_feature_matrix = filter_features(val_df)
print("Val Feature matrix shape (samples x unique bigrams):", val_feature_matrix.shape)

test_feature_matrix = filter_features(test_df)
print(
    "\nTest Feature matrix shape (samples x unique bigrams):",
    test_feature_matrix.shape,
)

## Model Implementation


In [ ]:
X_train = train_feature_matrix
y_train = train_df.label.values

X_val = val_feature_matrix
y_val = val_df.label.values

X_test = test_feature_matrix
y_test = test_df.label.values

#### Comet Logger

In [ ]:
experiment = Experiment(
    api_key="rwyMmTQC0QDIH0oF5XaSzgmh4",
    project_name="nlp-lr",
    workspace="youssefaboelwafa",
)
experiment.set_name(EXP_NAME)

### Training & Validation


In [ ]:
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)


class LR:
    def __init__(self, num_classes=5, epochs=100, learning_rate=0.01):
        self.num_classes = num_classes
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.weights = None
        self.bias = None

    def fit(self, X_train, y_train, X_val=None, y_val=None):
        n_samples, n_features = X_train.shape
        n_classes = self.num_classes

        self.weights = np.zeros((n_features, n_classes)).astype(np.float32)
        self.bias = np.zeros(n_classes).astype(np.float32)

        for epoch in range(self.epochs):
            print(f"Epoch {epoch + 1}/{self.epochs}")

            logits = X_train @ self.weights + self.bias
            probabilities = softmax(logits)

            output_hot_encoded = np.eye(n_classes)[y_train]

            error = probabilities - output_hot_encoded

            train_loss = (
                -np.sum(output_hot_encoded * np.log(probabilities + 1e-15)) / n_samples
            )

            train_preds = np.argmax(probabilities, axis=1)
            train_accuracy = np.mean(train_preds == y_train)

            experiment.log_metric("train_loss", train_loss, step=epoch)
            experiment.log_metric("train_accuracy", train_accuracy, step=epoch)

            dw = (1 / n_samples) * np.dot(X_train.T, error)
            db = (1 / n_samples) * np.sum(error, axis=0)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            if X_val is not None and y_val is not None:
                val_logits = X_val @ self.weights + self.bias
                val_probabilities = softmax(val_logits)
                val_output_hot_encoded = np.eye(n_classes)[y_val]

                val_loss = (
                    -np.sum(val_output_hot_encoded * np.log(val_probabilities + 1e-15))
                    / X_val.shape[0]
                )
                val_preds = np.argmax(val_probabilities, axis=1)
                val_accuracy = np.mean(val_preds == y_val)

                experiment.log_metric("val_loss", val_loss, step=epoch)
                experiment.log_metric("val_accuracy", val_accuracy, step=epoch)

        print("Training completed")
        print("Train loss: ", train_loss)
        print("Train accuracy: ", train_accuracy)
        if X_val is not None and y_val is not None:
            print("Val loss: ", val_loss)
            print("Val accuracy: ", val_accuracy)

    def predict(self, X):
        scores = np.dot(X, self.weights) + self.bias
        probabilities = softmax(scores)
        preds = np.argmax(probabilities, axis=1)
        return preds

    def save_weights(self, weights_path, bias_path):
        np.save(weights_path, self.weights)
        np.save(bias_path, self.bias)

    def load_weights(self, weights_path, bias_path):
        self.weights = np.load(weights_path)
        self.bias = np.load(bias_path)

In [ ]:
model = LR()
# model.fit(X_train, y_train, X_val, y_val)
model.load_weights("weights.npy", "bias.npy")

### Inference


In [ ]:
y_pred = model.predict(X_test)
test_accuracy = np.mean(y_pred == y_test)
print("Test accuracy: ", test_accuracy)

### Classification Report

In [ ]:
results = classification_report(y_test, y_pred, target_names=["0", "1", "2", "3", "4"])
print(results)

## Scikit-learn


### Logistic Regression


In [ ]:
logistic_model = LogisticRegression(max_iter=EPOCHS)

logistic_model.fit(X_train, y_train)

y_pred = logistic_model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy : ", test_accuracy)

#### Classification Report

In [ ]:
results = classification_report(y_test, y_pred, target_names=["0", "1", "2", "3", "4"])
print(results)

### Stochastic Gradient Descent


In [ ]:
sgd_sklearn = SGDClassifier(
    loss="log_loss", max_iter=EPOCHS, learning_rate="constant", eta0=0.05, shuffle=True
)

sgd_sklearn.fit(X_train, y_train)

y_pred = sgd_sklearn.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy : ", test_accuracy)

#### Classification Report

In [ ]:
results = classification_report(y_test, y_pred, target_names=["0", "1", "2", "3", "4"])
print(results)